# Averaging over a heatbath.

## Objective

Let our system initially be in the state

$$ |\varphi\rangle = \big(\alpha\ |0\rangle + \beta\ |1\rangle\big) \otimes |\phi\rangle $$

where the first qubit indicates the state of our measurement apparatus, and $|\phi\rangle$ indicates the $N$-qubit state of our heatbath.

We will sample a unitary $U$ from some distribution, and evolve to $|\varphi'\rangle = U |\varphi\rangle$.
This state can be written uniquely as a sum over computational-basis states:

$$ |\varphi'\rangle = c_0 |00\dots00\rangle + c_1 |00\dots01\rangle + \cdots + c_{2^{N+1}-1} |11\dots11\rangle $$

Let us compare each pair of states in the above expansion that share a state of the heatbath (that is, differ in only the apparatus state), and ask if the member of the pair with the apparatus in state $|0\rangle$ or $|1\rangle$ has more amplitude. That is to say, we ask for how many $0 \le i < 2^N$ is $|c_i| < |c_{i+2^N}|$.

The goal is to recover Born rule style statistics, where decoherence with the environment splits up the the state into approximately $|\alpha^2|$-many states (in the computational basis) where the apparatus state $|0\rangle$ has more amplitude, and $|\beta^2|$-many where $|1\rangle$ has more.

## Sampling a uniform unitary

Because the space $U(n)$ of unitary $n \times n$ matrices is a compact topological group, by [Haar's theorem](https://en.wikipedia.org/wiki/Haar_measure) we get a unique nice translation-invariant probability measure on $U(n)$.
[It is known](https://case.edu/artsci/math/esmeckes/Meckes_SAMSI_Lecture2.pdf) that we can sample from this space uniformly by selecting columns of a matrix one at a time, each uniformly normalized in $\mathbb{C}^n$, and orthogonal to all previous columns.
To select a vector uniformly from an orthogonal complement $S^\bot \subset \mathbb{C}^n$ it suffices to simply project out the components that lie in $S$.
This is precisely what Gram-Schmidt achieves, and therefore we may use numpy's QR decomposition to yield a uniform unitary matrix, as the Q component is simply a Gram-Schmidt reduction of the input.

Specifically, we start by generating $A \in \mathbb{C}^{n \times n}$ by sampling each entry from a complex normal.
This yields columns that are $n$-dimensionally complex normal, as the multivariate normal distribution has independent normal marginals.
Then we QR decompose $A$, and return the unitary component of the decomposition.

In [1]:
import math, cmath, random
import numpy as np

def uniformly_random_unitary(N):
    a = np.random.randn(N, N) + 1j * np.random.randn(N, N)
    q, _ = np.linalg.qr(a, mode="complete")
    # Verify unitarity explicitly.
    assert np.allclose(q.dot(q.conj().T), np.eye(N)), "Unitarity failure!"
    return q

## Experiment

Our experiment proceeds in six steps:

1. Given the coefficient $\alpha$ we generate a $\beta$ with random phase relative to $\alpha$ satisfying $|\alpha|^2 + |\beta|^2 = 1$.

2. We generate a uniformly random normalized state $|\phi\rangle$ for the heatbath.

3. We assemble the apparatus state and heatbath state into our overall state $\big(\alpha\ |0\rangle + \beta\ |1\rangle\big) \otimes |\phi\rangle$.

4. We generate a uniformly random unitary interaction $U \in U(2^{N+1})$.

5. We evolve our overall state by $U$.

6. We count up the number of heatbath states for which the apparatus measures more 0ey than 1ey.

In [2]:
def uniformly_random_state(N):
    a = np.random.randn(N) + 1j * np.random.randn(N)
    a /= np.linalg.norm(a)
    return a

def run(heatbath_bits, alpha):
    # Step 1: Compute a randomly phased beta.
    beta = cmath.exp(1j * random.uniform(0, 2*math.pi)) * (1 - alpha**2.0)**0.5
    apparatus_state = np.array([alpha, beta])
    # Assert that our state is L2 normalized.
    assert np.allclose(np.linalg.norm(apparatus_state), 1)

    # Step 2: Choose a random state for the heatbath.
    heatbath = uniformly_random_state(2**heatbath_bits)

    # Step 3: Our overall state is the tensor product of the above two.
    overall_state = np.kron(apparatus_state, heatbath)

    # Step 4: Generate a random interaction.
    evolution = uniformly_random_unitary(2**(heatbath_bits + 1))

    # Maybe: Consider only partial evolution? This is a different measure on unitaries...
    #import scipy.linalg
    #evolution = scipy.linalg.fractional_matrix_power(evolution, 0.1)

    # Step 5: Interact the apparatus with the heatbath.
    overall_state = evolution.dot(overall_state)

    # Step 6: Count up the number of states where alpha wins.
    norms = np.abs(overall_state.reshape((2, -1)))
    alpha_win_proportion = np.average(norms[0] < norms[1])

    return alpha_win_proportion

We now proceed to do a bunch of random trials.

In [3]:
print np.average([run(heatbath_bits=3, alpha=0.3) for _ in xrange(10000)])

0.501975


wat

Well, the unitary matrix just completely scrambled the state, making everything uniform.
What was it we wanted to do again?
Were we being dumb all along?